# AgentKit SDK Memory Tutorial 01: Memory 全链路使用教程

本教程可以**单独使用**，目标是在一个 Notebook 中完整跑通：

- 从零创建一个示例 Agent 项目；
- 为 Agent 集成长期记忆组件 `LongTermMemory`；
- 使用 Memory SDK 创建 / 查询 / 获取连接信息；
- 使用 `sdk.bind_memory_env_to_config_for_veadk` 自动将连接信息写入 `agentkit.yaml`；
- 使用 `AgentKitClient.launch` + `invoke` 验证 Memory 是否生效；
- 使用 Memory SDK 更新与删除 Memory 资源。

在开始本教程之前，请确保你已经**正确配置了凭证**。AgentKit 需要两类凭证：

### 第一类：SDK 与 CLI 的云服务访问凭证
用于创建和管理云资源（CR、CP、TOS、Runtime 等）。推荐配置方式：

```bash
# 使用 agentkit 命令行工具全局配置（推荐）
agentkit config --global --set volcengine.access_key=YOUR_ACCESS_KEY
agentkit config --global --set volcengine.secret_key=YOUR_SECRET_KEY
agentkit config --global --set volcengine.region=cn-beijing
```

### 第二类：Agent 运行时的环境变量
用于 Agent 访问大模型等服务。在 `agentkit.yaml` 的 `runtime_envs` 中配置(Cloud/Hybrid模式无需配置，Local模式需配置)：

```python
# 模型访问凭证（示例）
config.add_runtime_env("MODEL_AGENT_NAME", "your-model-name")
config.add_runtime_env("MODEL_AGENT_API_KEY", "your-model-api-key")
```

> **注意**：如果你还没有配置凭证，请先学习 `00_configuration_and_credentials.ipynb` 教程完成配置。

额外依赖：`mem0ai==0.1.118`（用于 Mem0 后端）。

你不需要预先准备任何工程或 Memory 资源，本教程会通过 SDK 完成全链路操作。

## 0. 导入依赖并准备项目

本节我们将：
- 导入 `Path`、`logging` 等标准库；
- 导入 Starter Toolkit SDK：`agentkit.toolkit import sdk`；
- 导入 `AgentConfig` 和 `AgentKitClient`；
- 准备一个示例项目目录 `tutorial_projects/t01-memory-agent`，如不存在则用 `sdk.init_project` 创建 basic 模板。


In [ ]:
from pathlib import Path
import logging

from agentkit.toolkit import sdk
from agentkit.toolkit.sdk import AgentConfig, AgentKitClient
from agentkit.toolkit.models import PreflightMode
from agentkit.toolkit.reporter import LoggingReporter

from agentkit.sdk.memory import (
    AgentkitMemoryClient,
    CreateMemoryCollectionRequest,
    ListMemoryCollectionsRequest,
    GetMemoryCollectionRequest,
    GetMemoryConnectionInfoRequest,
    UpdateMemoryCollectionRequest,
    DeleteMemoryCollectionRequest,
)

print("AgentKit SDK 与 Memory SDK 已导入。")

project_root = Path("tutorial_projects/t01-memory-agent")
project_name = "memory_agent"

print(f'约定项目路径: {project_root.resolve()}')

if not project_root.exists():
    print("项目目录不存在，使用 sdk.init_project 创建示例项目...")
    init_result = sdk.init_project(
        project_name=project_name,
        template="basic",
        project_root=str(project_root),
    )
    print("初始化是否成功:", init_result.success)
    print("项目路径:", init_result.project_path)
else:
    print("项目目录已存在，直接复用。")

config_file = project_root / "agentkit.yaml"
print("配置文件路径:", config_file)


## 1. 使用 AgentConfig 加载配置

`AgentConfig` 是管理 `agentkit.yaml` 的推荐方式。这里我们先加载配置，并简单查看 `launch_type`、`entry_point` 和当前的 `runtime_envs`。本 Notebook 会在需要时自动更新这些配置。

In [ ]:
config = AgentConfig.load(project_root)

print("当前配置概要:")
print(config)

print("launch_type:", config.launch_type)
print("agent_name:", config.agent_name)
print("entry_point:", config.entry_point)


## 1.1 Agent 端如何集成 LongTermMemory

上面的步骤只是准备好了项目和 `agentkit.yaml`，要让 Agent 真正“有记忆”，还需要在 **Agent 代码里集成长期记忆组件**。

如果你是从 `basic` 模板创建的项目，可以参考 Memory 文档中的示例，对入口文件（`entry_point`，通常是 `<project_name>.py` 或类似文件）做如下修改：

```python
import logging
import os

from veadk import Agent, Runner
from veadk.memory.long_term_memory import LongTermMemory

from agentkit.apps import AgentkitSimpleApp

logger = logging.getLogger(__name__)

app_name = "memory_agent"
app = AgentkitSimpleApp()

# 选择长期记忆后端："mem0" 或 "viking_mem"
backend = os.getenv("AGENTKIT_LONG_TERM_MEMORY_BACKEND", "mem0")

# 对于 viking_mem，需要从环境变量中读取 collection 名称；mem0 下 index 可以是任意字符串
long_term_memory = None
if backend == "viking_mem":
    collection_name = os.getenv("DATABASE_VIKINGMEM_COLLECTION")
    if not collection_name:
        raise ValueError("DATABASE_VIKINGMEM_COLLECTION environment variable is not set")
    long_term_memory = LongTermMemory(backend="viking_mem", index=collection_name)
elif backend == "mem0":
    if os.getenv("DATABASE_MEM0_API_KEY") is None:
        raise ValueError("MEM0_API_KEY is not set")
    if os.getenv("DATABASE_MEM0_BASE_URL") is None:
        raise ValueError("DATABASE_MEM0_BASE_URL is not set")
    long_term_memory = LongTermMemory(backend="mem0", app_name=app_name)

agent = Agent(
    name="simple_app_agent",
    instruction="You are a helpful assistant.",
    model_name="deepseek-v3-1-250821",
    long_term_memory=long_term_memory,
)
runner = Runner(agent=agent, app_name=app_name)


@app.entrypoint
async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    user_id = headers["user_id"]
    session_id = headers["session_id"]

    logger.info(
        f"Running agent with prompt: {prompt}, user_id: {user_id}, session_id: {session_id}"
    )
    response = await runner.run(
        messages=prompt,
        user_id=user_id,
        session_id=session_id,
    )

    logger.info(f"Run response: {response}")

    # save the teaching prompt and answer in long term memory
    await runner.save_session_to_long_term_memory(
        session_id=session_id,
        user_id=user_id,
    )

    return response


@app.ping
def ping() -> str:
    return "pong!"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)
```

> 提示：
> - 当 backend="mem0" 时，Mem0 的连接信息将通过环境变量 `DATABASE_MEM0_BASE_URL` / `DATABASE_MEM0_API_KEY` 传入（需要安装 `mem0ai`），后续我们会在本 Notebook 中用 `AgentConfig` 或 Memory SDK 来写入这些 env。
> - 当 backend="viking_mem" 时，需要配置 `DATABASE_VIKINGMEM_COLLECTION` / `DATABASE_VIKINGMEM_MEMORY_TYPE`。
>
> 你可以手动将上述代码拷贝到你的入口文件中；如果只是为了跑通本教程，也可以直接运行下一个代码单元，让 Notebook 自动在示例项目中生成一份带长期记忆能力的入口文件。完成后，再继续下面的步骤为不同 backend 写入对应的环境变量。

In [ ]:
# 自动为示例项目生成一个带 LongTermMemory 的入口文件，并更新 entry_point
from textwrap import dedent
import os

agent_file = project_root / "memory_agent.py"

memory_agent_code = dedent(
    '''\
    """simple memory_agent demo app with LongTermMemory

    Before running, please make sure you have set the following environment variables; otherwise, runtime errors may occur:

    - MODEL_AGENT_NAME      # model id in Volcano Engine Ark platform
    - MODEL_AGENT_API_KEY   # model api key in Volcano Engine Ark platform

    For Mem0 backend, you also need:
    - DATABASE_MEM0_BASE_URL
    - DATABASE_MEM0_API_KEY

    For Viking backend, you also need:
    - DATABASE_VIKINGMEM_COLLECTION
    - DATABASE_VIKINGMEM_MEMORY_TYPE
    """

    import logging
    import os

    from veadk import Agent, Runner
    from veadk.memory.long_term_memory import LongTermMemory

    from agentkit.apps import AgentkitSimpleApp

    logger = logging.getLogger(__name__)

    app_name = "memory_agent"
    app = AgentkitSimpleApp()

    # 选择长期记忆后端："mem0" 或 "viking_mem"
    backend = os.getenv("AGENTKIT_LONG_TERM_MEMORY_BACKEND", "mem0")

    # 对于 viking_mem，需要从环境变量中读取 collection 名称；mem0 下 index 可以是任意字符串
    long_term_memory = None
    if backend == "viking_mem":
        collection_name = os.getenv("DATABASE_VIKINGMEM_COLLECTION")
        if not collection_name:
            raise ValueError("DATABASE_VIKINGMEM_COLLECTION environment variable is not set")
        long_term_memory = LongTermMemory(backend="viking_mem", index=collection_name)
    elif backend == "mem0":
        if os.getenv("DATABASE_MEM0_API_KEY") is None:
            raise ValueError("MEM0_API_KEY is not set")
        if os.getenv("DATABASE_MEM0_BASE_URL") is None:
            raise ValueError("DATABASE_MEM0_BASE_URL is not set")
        long_term_memory = LongTermMemory(backend="mem0", app_name=app_name)

    agent = Agent(
        name="simple_app_agent",
        instruction="You are a helpful assistant.",
        model_name="deepseek-v3-1-250821",
        long_term_memory=long_term_memory,
    )
    runner = Runner(agent=agent, app_name=app_name)


    @app.entrypoint
    async def run(payload: dict, headers: dict) -> str:
        prompt = payload["prompt"]
        user_id = headers["user_id"]
        session_id = headers["session_id"]

        logger.info(
            f"Running agent with prompt: {prompt}, user_id: {user_id}, session_id: {session_id}"
        )
        response = await runner.run(
            messages=prompt,
            user_id=user_id,
            session_id=session_id,
        )

        logger.info(f"Run response: {response}")

        # save the teaching prompt and answer in long term memory
        await runner.save_session_to_long_term_memory(
            session_id=session_id,
            user_id=user_id,
        )

        return response


    @app.ping
    def ping() -> str:
        return "pong!"


    if __name__ == "__main__":
        app.run(host="0.0.0.0", port=8000)
    '''
)

# 写入示例代码文件
os.makedirs(agent_file.parent, exist_ok=True)
with open(agent_file, "w", encoding="utf-8") as f:
    f.write(memory_agent_code)

# 更新 entry_point 指向新文件
config.entry_point = agent_file.name
config.save()

print("已生成带长期记忆能力的示例 Agent 代码:", agent_file)
print("当前 entry_point:", config.entry_point)

# 可选：为 Mem0 添加依赖到 requirements.txt（如果尚未添加）
requirements_file = project_root / "requirements.txt"
if requirements_file.exists():
    content = requirements_file.read_text(encoding="utf-8")
    if "mem0ai" not in content:
        requirements_file.write_text(content.rstrip() + "\nmem0ai==0.1.118\n", encoding="utf-8")
        print("已在 requirements.txt 中添加 mem0ai 依赖。")


## 2. 使用 Memory SDK 创建记忆库

接下来我们通过 Memory SDK 创建一个新的记忆库（Memory Collection）。

这里我们以 Mem0 类型为例：

- 使用 `CreateMemoryCollectionRequest` 指定 `name`、`description`、`provider_type`；
- 调用 `AgentkitMemoryClient.create_memory_collection` 创建记忆库；
- 保存返回的 `MemoryId`，供后续查询、绑定和管理使用。

In [ ]:
import uuid
from agentkit.sdk.memory.types import LongTermForCreateMemoryCollection, LongTermStrategiesItemForCreateMemoryCollection
# 创建 Memory SDK 客户端
memory_client = AgentkitMemoryClient()

# 通过 SDK 创建一个新的 Mem0 记忆库
create_req = CreateMemoryCollectionRequest(
    name="tutorial_memory",
    description="Memory created from tutorial",
    provider_type="MEM0",  # 也可以是 "VIKINGDB_MEMORY"
    long_term_configuration=LongTermForCreateMemoryCollection(
        strategies=[
            LongTermStrategiesItemForCreateMemoryCollection(
                name="memory_tutorial_strategy" + uuid.uuid4().hex[:6],
                type="Summary",
            ),
            LongTermStrategiesItemForCreateMemoryCollection(
                name="memory_tutorial_strategy" + uuid.uuid4().hex[:6],
                type="Semantic",
            ),
            LongTermStrategiesItemForCreateMemoryCollection(
                name="memory_tutorial_strategy" + uuid.uuid4().hex[:6],
                type="UserPreference",
            )
        ]
    )
)

create_resp = memory_client.create_memory_collection(create_req)
MEMORY_ID = create_resp.memory_id

print("已通过 SDK 创建 Memory：")
print("  MemoryId:", MEMORY_ID)
print("  ProviderType:", create_resp.provider_type)
print("  Status:", create_resp.status)


## 3. 查询 Memory 列表与详情

我们可以通过 Memory SDK 查询当前账号下的记忆库列表，以及某个记忆库的详细信息：

- `ListMemoryCollectionsRequest`：分页列出 Memory；
- `GetMemoryCollectionRequest`：按 `MemoryId` 获取单个 Memory 的详情（类型、策略等）。

In [ ]:
# 列出部分记忆库
list_req = ListMemoryCollectionsRequest(page_number=1, page_size=10)
list_resp = memory_client.list_memory_collections(list_req)

memories = list_resp.memories or []
print(f"共返回 {len(memories)} 条记忆库记录 (仅展示前 10 条)。")
for m in memories:
    print(
        f"MemoryId={m.memory_id}, Name={m.name}, ProviderType={m.provider_type}, Status={m.status}"
    )

# 获取刚刚创建的 Memory 的详情
get_req = GetMemoryCollectionRequest(memory_id=MEMORY_ID)
get_resp = memory_client.get_memory_collection(get_req)

print("\n当前教程使用的 Memory 详情：")
print("  MemoryId:", get_resp.memory_id)
print("  ProviderType:", get_resp.provider_type)
print("  Status:", get_resp.status)
print("  LongTermStrategies:", getattr(get_resp.long_term_configuration, "strategies", None))


## 4. 获取连接信息并自动绑定到 agentkit.yaml

为了让 Agent 在 Runtime 中实际使用 Memory，需要将 Memory 的连接信息写入 `agentkit.yaml` 中的 `launch_types.<strategy>.runtime_envs`。

你可以手动拼接这些环境变量，但更推荐使用：

- `GetMemoryConnectionInfoRequest`：获取给定 `MemoryId` 的连接信息（BaseUrl / AuthKey 等）；
- `sdk.bind_memory_env_to_config_for_veadk(project_dir, memory_id, strategy_name)`：自动基于 `MemoryId` 查询连接信息并更新配置文件。

下面我们先简单看一眼连接信息的结构，然后调用 `bind_memory_env_to_config_for_veadk` 自动写入配置。

In [ ]:
import json
# 查看 Memory 连接信息（可选，仅用于理解数据结构）
conn_req = GetMemoryConnectionInfoRequest(memory_id=MEMORY_ID)
conn_resp = memory_client.get_memory_connection_info(conn_req)
print("ConnectionInfos:")
print(conn_resp.model_dump_json(indent=2))


In [ ]:
# 使用 sdk.bind_memory_env_to_config_for_veadk 自动将连接信息写入 agentkit.yaml
updated_config = sdk.bind_memory_env_to_config_for_veadk(
    project_dir=project_root,
    memory_id=MEMORY_ID,
    strategy_name="cloud",
)

print("\nMemory 已绑定到配置。当前 cloud.runtime_envs:")
print(updated_config.get_strategy_config("cloud").get("runtime_envs"))


## 5. 使用 AgentKitClient.launch + invoke 验证 Memory

完成 Memory 绑定后，就可以像普通 Agent 一样：

1. 使用 `AgentKitClient.launch` 完成构建 + 部署；
2. 在第一次对话中写入一个“秘密”；
3. 在新的会话中提问，验证是否能从长期记忆中取回信息。

> 提示：长期记忆提取通常是异步的，建议两次调用之间等待一段时间。

In [ ]:
# 配置 logger 和 Reporter
logger = logging.getLogger("agentkit_memory_tutorial")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("[%(levelname)s] %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

reporter = LoggingReporter(logger=logger)

# 使用绑定了 Memory 的配置创建 AgentKitClient
client = AgentKitClient(config=updated_config, reporter=reporter)
print("已创建 AgentKitClient:", client)

# 1) 构建 + 部署
launch_result = client.launch(
    platform="auto",
    preflight_mode=PreflightMode.WARN,
)
print("[Client.launch] 是否成功:", launch_result.success, launch_result.error)

In [ ]:
# 2) 首轮对话：告诉 Agent 一个秘密
teaching_headers = {"user_id": "user-1", "session_id": "session-1"}
teach_result = client.invoke(
    payload={"prompt": "我喜欢吃草莓"},
    headers=teaching_headers,
)
print("[Client.invoke teaching] 成功:", teach_result.success)
print("响应:", teach_result.response)

# 建议此处手动等待一段时间，再运行下面的单元进行验证。

In [ ]:
# 3) 新会话中询问秘密，验证长期记忆是否生效
# 注意，记忆存储可能需要一些时间，可能需要等待几分钟才能生效
student_headers = {"user_id": "user-1", "session_id": "session-2"}
student_result = client.invoke(
    payload={"prompt": "推荐适合我的饭后水果"},
    headers=student_headers,
)
print("[Client.invoke student] 成功:", student_result.success)
print("响应内容:")
print(student_result.response)


## 6. 管理与删除 Memory

在实际项目中，你可能需要在不同阶段对 Memory 进行管理，例如：

- 更新描述等元信息；
- 在不再需要时删除 Memory 资源以释放配额。

下面示例演示如何通过 SDK：

- 使用 `UpdateMemoryCollectionRequest` 更新 Memory 描述；
- 使用 `DeleteMemoryCollectionRequest` 删除 Memory（可选执行）。

In [ ]:
# 更新 Memory 描述
update_req = UpdateMemoryCollectionRequest(
    memory_id=MEMORY_ID,
    description="Memory updated from Tutorial",
)
update_resp = memory_client.update_memory_collection(update_req)

print("已更新 Memory 描述：")
print("  MemoryId:", update_resp.memory_id)
print("  ProviderType:", update_resp.provider_type)
print("  LongTermConfiguration:", update_resp.long_term_configuration)

In [ ]:

# （可选）删除 Memory 资源
DO_DELETE_MEMORY = True  # 如需删除，请显式改为 True

if DO_DELETE_MEMORY:
    delete_req = DeleteMemoryCollectionRequest(memory_id=MEMORY_ID)
    delete_resp = memory_client.delete_memory_collection(delete_req)
    print("\n已删除 Memory：")
    print("  MemoryId:", delete_resp.memory_id)
    print("  Status:", delete_resp.status)
else:
    print("\n保留当前 Memory 资源，方便后续继续使用。将 DO_DELETE_MEMORY 设为 True 可体验删除流程。")


## 7. 小结

在本教程中，你使用 **AgentKit SDK + Memory SDK** 完成了 Memory 的全链路流程：

1. 通过 `sdk.init_project` 创建示例 Agent 项目；
2. 为 Agent 自动生成集成 `LongTermMemory` 的入口代码；
3. 使用 `AgentkitMemoryClient.create_memory_collection` 创建新的记忆库；
4. 使用 `ListMemoryCollections` / `GetMemoryCollection` 查询 Memory 列表与详情；
5. 使用 `GetMemoryConnectionInfo` 了解连接信息结构，并通过 `sdk.bind_memory_env_to_config_for_veadk` 自动写入 `agentkit.yaml`；
6. 使用 `AgentKitClient.launch` / `invoke` 在 Runtime 中验证长期记忆是否生效；
7. 使用 `UpdateMemoryCollection` / `DeleteMemoryCollection` 管理与删除 Memory 资源。

你可以在此基础上：

- 为不同项目或环境创建多套 Memory，并通过 SDK 在 CI/CD 中自动绑定；
- 结合 Knowledge / Tools 等能力，构建具备长期记忆的复杂智能体应用；
- 根据业务需要，自定义 Memory 策略与网络配置（详见 Memory OpenAPI 文档）。